In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers import GlobalMaxPooling1D, Bidirectional
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle

Using TensorFlow backend.


In [3]:
bird_audio_with_label = pickle.load( open( "cat.pkl", "r0.8218b" ) )
X_bird = bird_audio_with_label["cat"]
Y_bird = np.array([[1,0]] * X_bird.shape[0])
print X_bird.shape
print Y_bird.shape

bed_audio_with_label = pickle.load( open( "dog.pkl", "rb" ) )
X_bed = bed_audio_with_label["dog"]
Y_bed = np.array([[0,1]] * X_bed.shape[0])
print X_bed.shape
print Y_bed.shape

X = np.vstack((X_bird, X_bed))
Y = np.vstack((Y_bird, Y_bed))
print X.shape
print Y.shape

(1733, 20, 100)
(1733, 2)
(1746, 20, 100)
(1746, 2)
(3479, 20, 100)
(3479, 2)


In [32]:
inp = Input(shape=(20,100 ))
x = (LSTM(100, return_sequences=True))(inp)
x = (LSTM(100, return_sequences=True))(x)
#x = (LSTM(100, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.4)(x)

#x = Dense(20, activation = "sigmoid")(x)
#x = Dense(5, activation = "sigmoid")(x)

x = Dense(2, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 20, 100)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 100)           80400     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 161,002
Trainable params: 161,002
Non-trainable params: 0
_________________________________________________________________


In [33]:
file_path="RNN_val_loss_cat_dog.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=20)

Train on 3131 samples, validate on 348 samples
Epoch 1/20
3131/3131 [==============================] - 10s - loss: 0.6699 - acc: 0.5899 - val_loss: 0.9875 - val_acc: 0.2730
Epoch 2/20
3131/3131 [==============================] - 7s - loss: 0.6057 - acc: 0.6701 - val_loss: 0.6240 - val_acc: 0.6351
Epoch 3/20
3131/3131 [==============================] - 7s - loss: 0.5570 - acc: 0.7308 - val_loss: 0.8347 - val_acc: 0.4425
Epoch 4/20
3131/3131 [==============================] - 9s - loss: 0.4791 - acc: 0.7774 - val_loss: 0.5131 - val_acc: 0.7328
Epoch 5/20
3131/3131 [==============================] - 10s - loss: 0.4130 - acc: 0.8093 - val_loss: 0.6741 - val_acc: 0.6609
Epoch 6/20
3131/3131 [==============================] - 11s - loss: 0.3967 - acc: 0.8250 - val_loss: 0.5622 - val_acc: 0.7155
Epoch 7/20
3131/3131 [==============================] - 9s - loss: 0.3330 - acc: 0.8566 - val_loss: 0.5756 - val_acc: 0.7270
Epoch 8/20
3131/3131 [==============================] - 10s - loss: 0.2880 

In [34]:
model = load_model(file_path)

In [53]:
a= model.predict(X[3470:3471])

In [54]:
a

array([[ 0.02894742,  0.97105265]], dtype=float32)